In [1]:
from bs4 import BeautifulSoup, NavigableString
import requests
from time import sleep
from random import randint
import math
from tqdm import tqdm
import pickle
import pandas as pd
from datetime import date
from datetime import datetime
import json

import pymysql                        # for getting data from a SQL database
from sqlalchemy import create_engine, text  # for establishing the connection and authentication


from webscraper.scrap_modules import scraper_ds as mybib


%load_ext autoreload
%autoreload 2

### Importing keywords

In [2]:
keyword_list = mybib.import_keyword_list()
keyword_list

['Data%20Analyst']

### Scraping Prep
- connect with MySQL to get id's
- create empty DataFrame for saving scraping results
- print results for check

In [3]:
sql_data = mybib.connect_sql_database()
scraper_df = mybib.initialize_empty_df()

print("Overview SQL Database")
display(sql_data.head(5))
display(sql_data.tail(5))

print("Empty Dataframe")
display(scraper_df)

print("Overview ID control")
id_control = sql_data['id'].tolist()
id_control

Overview SQL Database


,id,title,company,city,posting_date,job_description,seniority_level,job_function,industries,scraping_date,url,keyword


,id,title,company,city,posting_date,job_description,seniority_level,job_function,industries,scraping_date,url,keyword


Empty Dataframe


,id,title,company,city,posting_date,job_description,seniority_level,job_function,industries,scraping_date,url,keyword


Overview ID control


[]

In [21]:
dataframe, dict = mybib.total_loop_rough_keywords(keyword_list, scraper_df, id_control)

[autoreload of webscraper.scrap_modules.scraper_ds failed: Traceback (most recent call last):
  File "C:\Users\Domen\AppData\Roaming\Python\Python310\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\Domen\AppData\Roaming\Python\Python310\site-packages\IPython\extensions\autoreload.py", line 475, in superreload
    module = reload(module)
  File "C:\Users\Domen\anaconda3\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 879, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1017, in get_code
  File "<frozen importlib._bootstrap_external>", line 947, in source_to_code
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "c:\Users\Domen\IronHack\01_projects\IH_final_project_data-jobs\webscraper\scrap_modules\scrape

Start creating backend links for Data Analyst. Total of found pages: 2

Start extracting ids from https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Data%20Analyst&location=Berlin%2C%20Berlin%2C%20Germany&locationId=&geoId=104944500&f_TPR=&distance=25&f_E=2%2C3%2C4&position=1&start=0
base-card div not found for this job.
I will sleep for 3.649 seconds.

Start extracting ids from https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Data%20Analyst&location=Berlin%2C%20Berlin%2C%20Germany&locationId=&geoId=104944500&f_TPR=&distance=25&f_E=2%2C3%2C4&position=1&start=25
I will sleep for 3.239 seconds.



Sleep 2162 seconds:  22%|█████████████▉                                                | 11/49 [00:01<00:04,  7.92it/s]

,id,title,company,city,posting_date,job_description,seniority_level,job_function,industries,scraping_date,url,keyword
0,3707596225,Senior Data Analyst (f/m/d),Clariness,"Berlin, Germany",1 week ago,"([Full-time; unlimited -], [Engaging with key ...",Mid-Senior level,Contract,"Technology, Information and Internet",2023-09-21,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst


Sleep 2819 seconds:  27%|████████████████▍                                             | 13/49 [00:05<00:10,  3.37it/s]

...

Sleep 2758 seconds:  98%|████████████████████████████████████████████████████████████▋ | 48/49 [01:38<00:02,  2.10s/it]

,id,title,company,city,posting_date,job_description,seniority_level,job_function,industries,scraping_date,url,keyword
35,3700488094,Business Intelligence Consultant (m/f/d) (2),Machine Learning Reply GmbH,"Berlin, Berlin, Germany",3 weeks ago,([Formulate business needs and translate needs...,Entry level,Full-time,IT Services and IT Consulting,2023-09-21,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst


Sleep 2758 seconds: 100%|██████████████████████████████████████████████████████████████| 49/49 [01:40<00:00,  2.06s/it]


,id,title,company,city,posting_date,job_description,seniority_level,job_function,industries,scraping_date,url,keyword
31,3724750965,Online Data Analyst,TELUS International,"Potsdam, Brandenburg, Germany",1 day ago,([Full Professional Proficiency in English and...,Entry level,Contract,IT Services and IT Consulting,2023-09-21,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst
32,3720129659,Data Engineer Specialist (m/f/d),Qcells Europe,"Berlin, Berlin, Germany",1 week ago,([You create and maintain optimal data pipelin...,Entry level,Full-time,Renewable Energy Semiconductor Manufacturing,2023-09-21,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst
33,3590312287,Data Center Engineer L2 (On Demand),DeRisk Technologies,"Berlin, Berlin, Germany",5 months ago,"([Servers (Virtual & Physical), Storage & Back...",Associate,Contract,"Technology, Information and Internet",2023-09-21,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst
34,3700038161,(Senior) Data Analyst (m/w/d),SIXT,"Berlin, Berlin, Germany",3 weeks ago,"([Du bist dafür verantwortlich, aus unterschie...",Mid-Senior level,Full-time,Travel Arrangements,2023-09-21,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst
35,3700488094,Business Intelligence Consultant (m/f/d) (2),Machine Learning Reply GmbH,"Berlin, Berlin, Germany",3 weeks ago,([Formulate business needs and translate needs...,Entry level,Full-time,IT Services and IT Consulting,2023-09-21,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst


I will sleep for 4.591 seconds.


In [28]:
display(dataframe)
display(dict)

,id,title,company,city,posting_date,job_description,seniority_level,job_function,industries,scraping_date,url,keyword
0,3707596225,Senior Data Analyst (f/m/d),Clariness,"Berlin, Germany",1 week ago,"([Full-time; unlimited -], [Engaging with key ...",Mid-Senior level,Contract,"Technology, Information and Internet",2023-09-21,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst
1,3694298306,(Senior) Data Analyst (m/f/d) - Customer Data,MOIA,"Berlin, Germany",1 month ago,([Analyze transactional data to identify trend...,Mid-Senior level,Full-time,"Technology, Information and Internet",2023-09-21,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst
2,3701510210,Data Analyst (w/m/d),Qualitas Energy Deutschland GmbH,"Berlin, Berlin, Germany",3 weeks ago,"([Eigenständiges Durchsuchen, Aufbereiten und ...",Entry level,Full-time,Financial Services,2023-09-21,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst
3,3723261476,BI Analyst,Scoutbee,"Berlin, Germany",1 day ago,([Data-driven & analytics mindset : Utilize a ...,Mid-Senior level,Full-time,"Technology, Information and Internet",2023-09-21,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst
4,3641771902,Senior Data Analyst (m/f/d),Pitch,"Berlin, Germany",3 months ago,([Inform and influence our growth and product ...,Mid-Senior level,Full-time,"Technology, Information and Internet",2023-09-21,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst
5,3582381366,Young Professional Client Data Analyst (m/f/d) ←,Talentschmiede Unternehmensberatung AG,"Berlin, Berlin, Germany",5 months ago,([Responsibility for the analysis of data stru...,Entry level,Full-time,IT Services and IT Consulting,2023-09-21,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst
6,3674803883,(Senior) Data Analyst (all genders),Native Instruments,"Berlin, Germany",2 months ago,"([Evaluate user experience, engagement and pur...",Mid-Senior level,Full-time,"Technology, Information and Internet",2023-09-21,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst
7,3708672280,Senior Data Analyst,Gemma Analytics,"Berlin, Berlin, Germany",3 weeks ago,([2 or more years of experience as an analyst ...,Mid-Senior level,Full-time,IT Services and IT Consulting,2023-09-21,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst
8,3713235511,Wissenschaftlicher Mitarbeiter / Data Analyst ...,Charité - Universitätsmedizin Berlin,"Berlin, Berlin, Germany",2 weeks ago,([Analyse von Massenzytometriedaten und statis...,Entry level,Full-time,Hospitals and Health Care,2023-09-21,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst
9,3720128757,DataAnalyst&SystemDeveloper (m/f/d),Qcells Europe,"Berlin, Berlin, Germany",1 week ago,([Sie interagieren und unterstützen unseren Re...,Entry level,Full-time,Renewable Energy Semiconductor Manufacturing,2023-09-21,https://www.linkedin.com/jobs/search?keywords=...,Data Analyst


None

In [ ]:
keywords_input = ['Data%20Analyst']
keywords = []


# with open(f"webscrap_data/{filename}", 'rb') as file:
#        scraper_df = pickle.load(file)

In [ ]:
# https://www.linkedin.com/jobs/search?keywords=%7Bkey%7D&location=Berlin%2C%20Berlin%2C%20Germany&locationId=&geoId=106967730&f_TPR=&f_PP=106967730&distance=25&f_JT=F&f_E=2%2C3%2C4&position=1&pageNum=0

    # first_url = f"https://www.linkedin.com/jobs/search?keywords={key}&location=Berlin%2C%20Berlin%2C%20Germany&locationId=&geoId=106967730&f_TPR=&distance=25&f_JT=F&f_E=2%2C3&f_PP=106967730&position=1&pageNum=0"

In [ ]:
first_url = f"https://www.linkedin.com/jobs/search?keywords=%7Bkey%7D&location=Berlin%2C%20Berlin%2C%20Germany&locationId=&geoId=106967730&f_TPR=&f_PP=106967730&distance=25&f_JT=F&f_E=2%2C3%2C4&position=1&pageNum=0"



In [ ]:

# soup_dict = {}
# links = {}
# for key in keywords:
#     # first_url = f"https://www.linkedin.com/jobs/search?keywords=%7Bkey%7D&location=Berlin%2C%20Berlin%2C%20Germany&locationId=&geoId=106967730&f_TPR=&f_PP=106967730&distance=25&f_JT=F&f_E=2%2C3%2C4&position=1&pageNum=0"
#     key_name = key.replace('%20', " ")
        
#     # response = requests.get(first_url)
#     # response.status_code # 200 status code means OK!
#     # soup = BeautifulSoup(response.content, "html.parser")
    
#     # number_of_results = soup.find('span', class_="results-context-header__job-count").text
#     # numb = int(number_of_results.replace(",", "").replace("+", ""))
#     # # print(numb) # number of job post found 
    
#     # backend_call_url_list = create_backend_links(first_url, numb, key_name)
#     # with open(f'{key_name}_backend_urls.pkl', 'wb') as file:
#     #     pickle.dump(backend_call_url_list, file)
    
#     # id_list = get_id_dict(backend_call_url_list)
    
#     with tqdm(total=len(id_list), desc="Starting") as pbar:
#         for id in id_list:
#             dynamic_text = f"Progressing id: {id}"
#             pbar.set_description(dynamic_text)
#             if id not in id_control:
#                 try:
#                     scraper_df, id_control, small_soup = get_all_job_information(key_name, scraper_df, id, id_control)
#                     wait_time = randint(1,3000)
#                     pbar.set_description(f"Sleep {wait_time} seconds")
#                     sleep(wait_time/1000)
                    
#                     soup_dict[id] = small_soup
#                 except:
#                     print(f"Error when scraping data from id: {id}, temp backups are created in temp_data.")
#                     scraper_df_json = scraper_df.to_json('dataframe.json', orient='split', date_format='iso', indent=4)
#                     with open(f'temp_data/back_up_df.json', 'wb') as file:
#                         json.dump(scraper_df_json, file=file)
#                     with open('temp_data/small_soup_backup.json', 'w') as f:
#                         json.dump(small_soup, f)
#                     with open('temp_data/id_list_backup.pkl', "wb") as file:
#                         pickle.dump(links, file=file)
#             else:
#                 pbar.set_description(f"Will skip {id} because is already in the dataset.")
#             pbar.update(1)
    
#     current_date = datetime.now()
#     time = current_date.strftime("%Y-%m-%d_%H-%M-%S")
#     time

#     with open(f'webscrap_data/webscrap{str(time)}.pkl', 'wb') as file:
#         pickle.dump(scraper_df, file=file)
#     display(scraper_df.tail(5))
#     wait_time = randint(1,10000)
#     print("I will sleep for " + str(wait_time/1000) + " seconds.")
#     sleep(wait_time/1000)


In [ ]:
# use zlib.compress put into dict with id 
# pickle dict 

response = requests.get(f'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/3717202675')
soup = BeautifulSoup(response.content, "html.parser")

In [24]:
soup.find("section",{"class":"show-more-less-html"}).text.strip()

"CrazyLabs is a top 5 mobile games publisher with over 6 Billion downloads to date, a worldwide leader in hyper-casual games development, distribution, and innovation.We’re looking for an experienced Game Data Analyst to work closely with the product teams to analyze performance metrics, create insights, and guide data-driven decisions.If you have Passion for Gaming and work in a dynamic fast paced environment then this position is for you! Here’s what we need, up for the challenge?Analyzing large, complex data sets representing the behavior of millions of players to address strategic and operational business questionsOptimization and maximization users monetizationRecommendations for segmented offers based on user segments and behaviorData mining, pattern recognition & correlation analysis and insight generationDefinition, measurement and evaluation of AB test Provide actionable recommendations from game data to shape features, game economy, and future product roadmapsBe involved in b